In [57]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import models
from PIL import Image
from torchvision.models import resnet50
import torch.optim as optim  # For optimization algorithms
from torch.optim.lr_scheduler import CyclicLR  # For learning rate scheduling

In [58]:
# Change with path to your dataset path, you may check the exact path using the files tab (folder icon in the sidebar)
train_dir = "/kaggle/input/vlg-recruitment-24-challenge/vlg-dataset/vlg-dataset/train"
test_dir = "/kaggle/input/vlg-recruitment-24-challenge/vlg-dataset/vlg-dataset/test"

# Data Preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

full_train_dataset = ImageFolder(root=train_dir, transform=transform)


# Validate Class to Index Mapping
print("Class to Index Mapping:",full_train_dataset.class_to_idx)

Class to Index Mapping: {'antelope': 0, 'bat': 1, 'beaver': 2, 'blue+whale': 3, 'bobcat': 4, 'buffalo': 5, 'chihuahua': 6, 'cow': 7, 'dalmatian': 8, 'deer': 9, 'dolphin': 10, 'elephant': 11, 'german+shepherd': 12, 'giant+panda': 13, 'giraffe': 14, 'grizzly+bear': 15, 'hamster': 16, 'hippopotamus': 17, 'humpback+whale': 18, 'killer+whale': 19, 'leopard': 20, 'lion': 21, 'mole': 22, 'mouse': 23, 'otter': 24, 'ox': 25, 'persian+cat': 26, 'pig': 27, 'polar+bear': 28, 'raccoon': 29, 'rat': 30, 'seal': 31, 'siamese+cat': 32, 'skunk': 33, 'spider+monkey': 34, 'tiger': 35, 'walrus': 36, 'weasel': 37, 'wolf': 38, 'zebra': 39}


In [59]:
# Define the size of the validation set (15% of the full training dataset)
validation_split = 0.15
validation_size = int(len(full_train_dataset) * validation_split)
training_size = len(full_train_dataset) - validation_size

# Splitting the dataset
training_dataset, validation_dataset = torch.utils.data.random_split(
    full_train_dataset,
    [training_size, validation_size],
    generator=torch.Generator().manual_seed(13)  # Manual Seed for reproducibility every time model is trained
)

In [60]:
# DataLoader for training data
train_loader = DataLoader(
    training_dataset,
    batch_size=32,  
    shuffle=True,    # Shuffles the data at every epoch
    num_workers=4    # Kaggle processing limit is 4
)

# DataLoader for validation data
val_loader = DataLoader(
    validation_dataset,
    batch_size=32,  
    shuffle=False,  # No need to shuffle validation data
    num_workers=4    
)

In [61]:
# Model Definition
class MultilabelClassifier(nn.Module):
    def __init__(self,  num_classes):
        super(MultilabelClassifier, self).__init__()
        self.base_model = models.resnet50(pretrained=True)
        self.base_model.fc = nn.Linear(self.base_model.fc.in_features,num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.base_model(x)
        return self.sigmoid(x)






In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(full_train_dataset.classes)
model = MultilabelClassifier(num_classes).to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()  # Binary Cross-Entropy Loss

# Define the optimizer to update the model's parameters during training

optimizer = optim.AdamW(model.parameters(), weight_decay=1e-4)


learning_rate_scheduler = CyclicLR(
    optimizer,
    base_lr=3e-5,  # Lower bound of the learning rate
    max_lr=3e-4,    # Upper bound of the learning rate
    step_size_up = 100,  # Number of iterations to increase the learning rate
    mode="triangular2",  # Shape of the learning rate cycle
    cycle_momentum=False  # Whether to cycle momentum
)

# Training loop
epochs = 5
best_val_f1 = 0.0

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {train_loss/len(train_loader):.4f}")

#validation phase
model.eval()
running_val_loss = 0.0
with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_val_loss += loss.item()

avg_train_loss = train_loss / len(train_loader)
avg_val_loss = running_val_loss / len(val_loader)
print(f"Epoch [{epoch + 1}/{epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")


Epoch [1/5], Loss: 3.3722
Epoch [2/5], Loss: 3.2780


In [ ]:
# Prediction
model.eval()
test_images = [f for f in os.listdir(test_dir) if f.endswith('.jpg')]
test_predictions = []

for img_name in test_images:
    img_path = os.path.join(test_dir, img_name)
    image = Image.open(img_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image)
        predicted_class = torch.argmax(outputs, dim=1).item()
        test_predictions.append((img_name, full_train_dataset.classes[predicted_class]))

# Save Predictions
submission = pd.DataFrame(test_predictions, columns=['image_id', 'class'])
submission.to_csv("/kaggle/working/predictions.csv", index=False)